In [9]:
import webscrape_pd_scrape_patheos as web
import requests
from bs4 import BeautifulSoup as BS
from tqdm import tqdm
from pprint import pprint
import db_interface_sqlite as db

In [6]:
def fetch_blogs():
    """[ ] Document for refactoring
    
    """
    url = 'http://www.patheos.com/blogs'
    response = requests.get(url)
    soup = BS(response.content, 'html.parser')

    blog_list = []

    for blog in soup.find_all('div', attrs={"class":"related-content clearfix related-content-sm decorated channel-list"}):
        blog_url1 = blog.find('a')
        blog_url2 = blog_url1['href']
        blog_list.append(blog_url2) 

    blog_lists = []
    
    for i in blog_list:
        split_url = i.rsplit("/")
        blog_name = split_url[3]
        blog_lists.append(blog_name)
    
    blog_dict = {i:[] for i in blog_lists}

    blog_cat_prefix = "https://www.patheos.com/"

    i = 0    
    for blog_urls in tqdm(blog_dict, desc='Fetch blog urls for each category'):
        if blog_urls:
            query_url = blog_cat_prefix + blog_urls
            #sleep(random.uniform(1, 3))
            subsub_blog = requests.get(query_url)
            soup2 = BS(subsub_blog.content, 'html.parser')
            for blog in soup2.find_all('div', attrs={"class":"author-info"}):
                for blog_url0 in blog.find_all('div', attrs={"class":"title"}):
                    blog_url1 = blog_url0.find('a')
                    blog_url2 = blog_url1['href']
                    blog_dict[blog_urls].append(blog_url2)

            i = i + 1
        else:
            continue
    return blog_dict

In [7]:
results = fetch_blogs()

Fetch blog urls for each category: 100%|██████████| 20/20 [00:08<00:00,  2.48it/s]


In [ ]:
pprint(results)

In [11]:
with db.database() as data:
    pass

In [ ]:
sqliteConnection = sqlite3.connect(Path.home() / "py_apps" / "_appdata" / "webscrape_patheos" / "patheos.db")
cursor = sqliteConnection.cursor()
print("Connected to SQLite")

sqliteConnection.create_function("TOTILECASE", 1, _toTileCase)
select_query = "SELECT TOTILECASE(name) FROM SqliteDb_developers where id = ?"
cursor.execute(select_query, (id,))
name = cursor.fetchone()
print("Developer Name is", name)
cursor.close()